In [34]:
import clip
import torch
import numpy as np
from tqdm import tqdm
import os
from PIL import Image
import pandas as pd

In [35]:
gender_code = { 0 : 'male', 1 : 'female'}
race_code = { 0 : 'white', 1 : 'black', 2 : 'asian', 3 : 'indian', 4 : 'others'}

In [36]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
device

'cpu'

`device` will indicate wich `CLIP` model to use depending on the available hardware. If there is a GPU, `cuda:0` will be used  or `cuda:1` if there are multiple GPUs. If there is not a GPU, `cpu` will be used.

In [37]:
race_labels = ['black', 'white', 'asian', 'indian']
race_tkns = ['A photo of a person of color ' + label for label in race_labels]
race_text = clip.tokenize(race_tkns).to(device)

In [38]:
sex_labels = ['male', 'female']
sex_tkns = ['A photo of a person of sex ' + label for label in sex_labels]
sex_text = clip.tokenize(sex_tkns).to(device)

`tkns` is the domain of possible values. `CLIP` model predicts for each image the most probable sentence from `tkns`, in this case. 

Initializing usefull variables for `CLIP` model application.

In [39]:
BATCH_SIZE = 100000

dir_path = r'/Users/hanselblanco/Documents/4to/ML/UTKFace/UTKFace'
ln = 0
photo_paths = os.listdir(dir_path)

for path in photo_paths:
    if os.path.isfile(os.path.join(dir_path, path)):
        ln += 1

In [40]:
for dirname, _, filenames in os.walk(dir_path):
    filename = filenames[0]
    splitted = filename.split('_')
    age = splitted[0]
    gender = splitted[1]
    race = splitted[2]

In [41]:
df = pd.DataFrame(filenames, columns = ['filename'] )
df['filepath'] = df.filename.apply(lambda x: dir_path + x)
df['gender'] = df.filename.apply(lambda x: gender_code[int(x.split('_')[1])])
df['race'] = df.filename.apply(lambda x: race_code[int(x.split('_')[-2])])
df.head()

,filename,filepath,gender,race
0,9_1_2_20161219204347420.jpg.chip.jpg,/Users/hanselblanco/Documents/4to/ML/UTKFace/U...,female,asian
1,36_0_1_20170117163203851.jpg.chip.jpg,/Users/hanselblanco/Documents/4to/ML/UTKFace/U...,male,black
2,86_1_0_20170120225751953.jpg.chip.jpg,/Users/hanselblanco/Documents/4to/ML/UTKFace/U...,female,white
3,26_1_0_20170116171048641.jpg.chip.jpg,/Users/hanselblanco/Documents/4to/ML/UTKFace/U...,female,white
4,1_1_2_20161219154612988.jpg.chip.jpg,/Users/hanselblanco/Documents/4to/ML/UTKFace/U...,female,asian


Executing `CLIP` model for `photos_to_analize` images from the dataset.

In [42]:
sex_results = []
race_results = []
photos_to_analize = 100

photos = []
for j in range(photos_to_analize):
    photos.append(Image.open(dir_path + '/' + photo_paths[j]))

for i in tqdm(range(0, ln, BATCH_SIZE)):
    images = [preprocess(photos[j]) for j in range(len(photos))]
    image_input = torch.tensor(np.stack(images)).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image_input)
        sex_logits_per_image, sex_logits_per_text = model(image_input, sex_text)
        race_logits_per_image, race_logits_per_text = model(image_input, race_text)
        
        # The softmax function takes the original confidence and applys a transform to make all the confidence add up to one
        sex_probs = sex_logits_per_image.softmax(dim=-1).cpu().numpy()
        race_probs = race_logits_per_image.softmax(dim=-1).cpu().numpy()
        
        sex_results.append(sex_probs)
        race_results.append(race_probs)

100%|██████████| 1/1 [00:05<00:00,  5.68s/it]


In [43]:
race_res = np.concatenate(race_results, axis=0)
race_choices = np.argmax(race_res, axis=1)

sex_res = np.concatenate(sex_results, axis=0)
sex_choices = np.argmax(sex_res, axis=1)

In [44]:
r_getlabel = lambda x:race_labels[x]
r_vgetlabel = np.vectorize(r_getlabel)
races = r_vgetlabel(race_choices)

s_getlabel = lambda x:sex_labels[x]
s_vgetlabel = np.vectorize(s_getlabel)
genders = s_vgetlabel(sex_choices)

`races` is the vector with predicted labels to add to each sentence from `race_tkns` for each image, ordered.

`genders` is the vector with predicted labels to add to each sentence from `sex_tkns` for each image, ordered.

In [45]:
df = df[:photos_to_analize]
df['predicted_gender'] = genders
df['predicted_race'] = races
df.head()

,filename,filepath,gender,race,predicted_gender,predicted_race
0,9_1_2_20161219204347420.jpg.chip.jpg,/Users/hanselblanco/Documents/4to/ML/UTKFace/U...,female,asian,female,asian
1,36_0_1_20170117163203851.jpg.chip.jpg,/Users/hanselblanco/Documents/4to/ML/UTKFace/U...,male,black,male,black
2,86_1_0_20170120225751953.jpg.chip.jpg,/Users/hanselblanco/Documents/4to/ML/UTKFace/U...,female,white,female,white
3,26_1_0_20170116171048641.jpg.chip.jpg,/Users/hanselblanco/Documents/4to/ML/UTKFace/U...,female,white,female,asian
4,1_1_2_20161219154612988.jpg.chip.jpg,/Users/hanselblanco/Documents/4to/ML/UTKFace/U...,female,asian,female,asian


# Detecting bias in `CLIP` results.

## Race (Disparate impact)

### Selection Rate (Positive results / N)

In [46]:
positive_whites, positive_blacks, positive_asians, positive_indians = 0, 0, 0, 0

total_whites, total_blacks, total_asians, total_indians = 0, 0, 0, 0

for i in range(photos_to_analize):
    data = photo_paths[i].split('_')
    race_number = int(data[2])
    match race_code[race_number]:
        case 'white':
            total_whites += 1
            if races[i] == 'white':
                positive_whites += 1
        case 'black':
            total_blacks += 1
            if races[i] == 'black':
                positive_blacks += 1
        case 'asian':
            total_asians += 1
            if races[i] == 'asian':
                positive_asians += 1
        case 'indian':
            total_indians += 1
            if races[i] == 'indian':
                positive_indians += 1
        case default:
            continue
                
whites_sr, blacks_sr, asians_sr, indians_sr = positive_whites/ total_whites, positive_blacks/ total_blacks, positive_asians/ total_asians, positive_indians/ total_indians

whites_sr, blacks_sr, asians_sr, indians_sr
    

(0.9523809523809523, 0.7619047619047619, 1.0, 0.8)

#### Disparate impact

In [47]:

# disparate impact ratio = underprivileged group SR / privileged group SR
disp_impact_b_w = blacks_sr/ whites_sr
disp_impact_b_a = asians_sr/ whites_sr
disp_impact_b_i = indians_sr/ whites_sr
disp_impact_b_w, disp_impact_b_a, disp_impact_b_i

(0.8, 1.05, 0.8400000000000001)

In [48]:
if disp_impact_b_w < 0.8:
    print('Disparate impact present in black group / white group')
if disp_impact_b_a < 0.8:
    print('Disparate impact present in asian group / white group')
if disp_impact_b_i < 0.8:
    print('Disparate impact present in indian group / white group')

## Sex (Disparate impact)

### Selection Rate (Positive results / N)

In [49]:
positive_males, positive_females = 0, 0

total_males, total_females = 0, 0

for i in range(photos_to_analize):
    data = photo_paths[i].split('_')
    gender_number = int(data[1])
    match gender_code[gender_number]:
        case 'male':
            total_males += 1
            if genders[i] == 'male':
                positive_males += 1
        case 'female':
            total_females += 1
            if genders[i] == 'female':
                positive_females += 1
                
males_sr, females_sr = positive_males/ total_males, positive_females/ total_females

males_sr, females_sr
    

(0.9310344827586207, 0.9047619047619048)

#### Disparate impact

In [50]:

# disparate impact ratio = underprivileged group SR / privileged group SR
disp_impact = females_sr / males_sr
disp_impact

0.9717813051146384

In [51]:
if disp_impact < 0.8:
    print('Disparate impact present in female group / male group')


## Sex (Equalized odds)

In [52]:
tp_males, tp_females, fn_males, fn_females, fp_males, fp_females = 0, 0, 0, 0, 0, 0

for i in range(photos_to_analize):
    data = photo_paths[i].split('_')
    gender_number = int(data[1])
    match gender_code[gender_number]:
        case 'male':
            if genders[i] == 'male':
                tp_males += 1
            else:
                fp_females += 1
                fn_males += 1 # False negative (wrong no male prediction, in this case, equal to female false positive)
        case 'female':
            if genders[i] == 'female':
                tp_females += 1
            else:
                fp_males += 1
                fn_females += 1
                
males_tpr, females_tpr = tp_males/ (tp_males + fn_males), tp_females/ (tp_females + fn_females)

males_fpr, females_fpr = fp_males/ (fp_males + fn_males), fp_females/ (fp_females + fn_females)


#### True Positive Rates

In [53]:
males_tpr, females_tpr

(0.9310344827586207, 0.9047619047619048)

In [54]:
if abs(males_tpr - females_tpr) < 0.05:
    print('Equalized odds')
else:
    print('Not equalized odds')
    print(abs(males_tpr - females_tpr))

Equalized odds


#### False Positive Rates

In [55]:
males_fpr, females_fpr

(0.5, 0.5)

In [56]:
if abs(males_fpr - females_fpr) < 0.05:
    print('Equalized odds')
else:
    print('Not equalized odds')
    print(abs(males_fpr - females_fpr))

Equalized odds
